In [1]:
import pandas as pd 

In [2]:
df = pd.read_csv('combined.csv')
df

C:\Users\Joshua Sam\AppData\Local\Temp\ipykernel_16896\410874955.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('combined.csv')


,latitude,longitude,blk_no,road_name,postal_code,address,cbd_dist,min_dist_mrt,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1.294801,103.854467,1,BEACH ROAD,189673.0,1 BEACH RD,1351.713661,143.728945,2015-01,KALLANG/WHAMPOA,3 ROOM,1,BEACH RD,04 TO 06,74.0,Improved,1979.0,388000.0,63
1,1.294801,103.854467,1,BEACH ROAD,189673.0,1 BEACH RD,1351.713661,143.728945,2016-03,KALLANG/WHAMPOA,3 ROOM,1,BEACH RD,07 TO 09,68.0,Improved,1979.0,350000.0,62
2,1.294801,103.854467,1,BEACH ROAD,189673.0,1 BEACH RD,1351.713661,143.728945,2017-04,KALLANG/WHAMPOA,3 ROOM,1,BEACH RD,07 TO 09,68.0,Improved,1979.0,340000.0,61 years 05 months
3,1.294801,103.854467,1,BEACH ROAD,189673.0,1 BEACH RD,1351.713661,143.728945,2017-05,KALLANG/WHAMPOA,3 ROOM,1,BEACH RD,07 TO 09,68.0,Improved,1979.0,322000.0,61 years 05 months
4,1.294801,103.854467,1,BEACH ROAD,189673.0,1 BEACH RD,1351.713661,143.728945,2017-06,KALLANG/WHAMPOA,3 ROOM,1,BEACH RD,07 TO 09,68.0,Improved,1979.0,370000.0,61 years 03 months
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232290,1.286814,103.828928,9B,BOON TIONG ROAD,163009.0,9B BOON TIONG RD,2525.251132,182.864524,2023-12,BUKIT MERAH,5 ROOM,9B,BOON TIONG RD,19 TO 21,112.0,Improved,2016.0,1400000.0,91 years 02 months
232291,1.286814,103.828928,9B,BOON TIONG ROAD,163009.0,9B BOON TIONG RD,2525.251132,182.864524,2024-03,BUKIT MERAH,4 ROOM,9B,BOON TIONG RD,25 TO 27,92.0,Model A,2016.0,1188000.0,90 years 11 months
232292,1.286814,103.828928,9B,BOON TIONG ROAD,163009.0,9B BOON TIONG RD,2525.251132,182.864524,2024-06,BUKIT MERAH,5 ROOM,9B,BOON TIONG RD,34 TO 36,112.0,Improved,2016.0,1588000.0,90 years 08 months
232293,1.286814,103.828928,9B,BOON TIONG ROAD,163009.0,9B BOON TIONG RD,2525.251132,182.864524,2024-07,BUKIT MERAH,4 ROOM,9B,BOON TIONG RD,10 TO 12,92.0,Model A,2016.0,1100000.0,90 years 07 months


In [4]:
# Convert 'postal_code' to int64
df['postal_code'] = pd.to_numeric(df['postal_code'], errors='coerce').fillna(0).astype('int64')

# Convert 'lease_commence_date' to int64
df['lease_commence_date'] = pd.to_numeric(df['lease_commence_date'], errors='coerce').fillna(0).astype('int64')

# Extract lease remaining years from 'remaining_lease' and create 'lease_remain_years' as int64
def extract_lease_years(remaining_lease):
    try:
        years = int(remaining_lease.split(' ')[0])  # Extract the number of years
        return years
    except:
        return 0  # Return 0 for invalid or missing values

df['lease_remain_years'] = df['remaining_lease'].apply(extract_lease_years)

# Ensure 'lease_remain_years' is int64
df['lease_remain_years'] = df['lease_remain_years'].astype('int64')

# Verify the data types
print(df.dtypes)


latitude               float64
longitude              float64
blk_no                  object
road_name               object
postal_code              int64
address                 object
cbd_dist               float64
min_dist_mrt           float64
month                   object
town                    object
flat_type               object
block                   object
street_name             object
storey_range            object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
resale_price           float64
remaining_lease         object
lease_remain_years       int64
dtype: object


In [5]:
# Fill categorical columns with 'Unknown'
categorical_columns = ['blk_no', 'road_name', 'address']
for col in categorical_columns:
    df[col].fillna('Unknown', inplace=True)

# Fill numeric columns with the median
numeric_columns = ['latitude', 'longitude', 'cbd_dist', 'min_dist_mrt']
for col in numeric_columns:
    df[col].fillna(df[col].median(), inplace=True)

# Fill 'address' column where all values are missing
if df['address'].isnull().all():
    df['address'] = 'Unavailable'

# Handle storey_range (assuming "X TO Y" format is typical)
df['storey_range'].fillna('0 TO 0', inplace=True)

# For remaining columns, fill missing values with mode or appropriate strategy
remaining_columns = ['month', 'town', 'flat_type', 'block', 'street_name',
                     'floor_area_sqm', 'flat_model', 'lease_commence_date',
                     'resale_price', 'remaining_lease']

for col in remaining_columns:
    df[col].fillna(df[col].mode()[0], inplace=True)

# Verify missing values are handled
print(df.isnull().sum())

C:\Users\Joshua Sam\AppData\Local\Temp\ipykernel_16896\2925597691.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna('Unknown', inplace=True)
C:\Users\Joshua Sam\AppData\Local\Temp\ipykernel_16896\2925597691.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For ex

latitude               0
longitude              0
blk_no                 0
road_name              0
postal_code            0
address                0
cbd_dist               0
min_dist_mrt           0
month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
resale_price           0
remaining_lease        0
lease_remain_years     0
dtype: int64


In [6]:
import statistics

def get_median(x):
    if isinstance(x, str):  # Ensure the value is a string
        try:
            split_list = x.split(' TO ')
            float_list = [float(i) for i in split_list]
            median = statistics.median(float_list)
            return median
        except (ValueError, IndexError):
            return None  # Return None for invalid formats
    return None  # Return None for non-string values

# Apply the function to the column
df['storey_median'] = df['storey_range'].apply(lambda x: get_median(x))


In [7]:
# Extract the numeric value before 'years' and handle invalid values
def extract_years(remaining_lease):
    try:
        # Extract the number before 'years'
        if 'years' in remaining_lease:
            return int(remaining_lease.split(' years')[0])
        else:
            return int(remaining_lease)  # Handle cases where only the number is given
    except:
        return 0  # Default to 0 for invalid or missing values

# Apply the function to the 'remaining_lease' column and create 'lease_remain_years'
df['lease_remain_years'] = df['remaining_lease'].apply(extract_years)

# Drop the original 'remaining_lease' column
df.drop(columns=['remaining_lease'], inplace=True)

# Verify the changes
print(df[['lease_remain_years']].head())


   lease_remain_years
0                  63
1                  62
2                  61
3                  61
4                  61


In [8]:
scope_df = df[['cbd_dist','min_dist_mrt','floor_area_sqm','lease_remain_years','storey_median','resale_price']]
scope_df

,cbd_dist,min_dist_mrt,floor_area_sqm,lease_remain_years,storey_median,resale_price
0,1351.713661,143.728945,74.0,63,5.0,388000.0
1,1351.713661,143.728945,68.0,62,8.0,350000.0
2,1351.713661,143.728945,68.0,61,8.0,340000.0
3,1351.713661,143.728945,68.0,61,8.0,322000.0
4,1351.713661,143.728945,68.0,61,8.0,370000.0
...,...,...,...,...,...,...
232290,2525.251132,182.864524,112.0,91,20.0,1400000.0
232291,2525.251132,182.864524,92.0,90,26.0,1188000.0
232292,2525.251132,182.864524,112.0,90,35.0,1588000.0
232293,2525.251132,182.864524,92.0,90,11.0,1100000.0


In [9]:
scope_df = scope_df.drop_duplicates()
scope_df

,cbd_dist,min_dist_mrt,floor_area_sqm,lease_remain_years,storey_median,resale_price
0,1351.713661,143.728945,74.0,63,5.0,388000.0
1,1351.713661,143.728945,68.0,62,8.0,350000.0
2,1351.713661,143.728945,68.0,61,8.0,340000.0
3,1351.713661,143.728945,68.0,61,8.0,322000.0
4,1351.713661,143.728945,68.0,61,8.0,370000.0
...,...,...,...,...,...,...
232290,2525.251132,182.864524,112.0,91,20.0,1400000.0
232291,2525.251132,182.864524,92.0,90,26.0,1188000.0
232292,2525.251132,182.864524,112.0,90,35.0,1588000.0
232293,2525.251132,182.864524,92.0,90,11.0,1100000.0


In [1]:
!pip install seaborn

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached matplotlib-3.10.0-cp310-cp310-win_amd64.whl.metadata (11 kB)
  Using cached contourpy-1.3.1-cp310-cp310-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.55.3-cp310-cp310-win_amd64.whl.metadata (168 kB)
  Using cached pillow-11.0.0-cp310-cp310-win_amd64.whl.metadata (9.3 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Using cached matplotlib-3.10.0-cp310-cp310-win_amd64.whl (8.0 MB)
Using cached contourpy-1.3.1-cp310-cp310-win_amd64.whl (218 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
Using cached fonttools-4.55.3-cp310-cp310-win_amd64.whl (2.2 MB)
Using cached pillow-11.0.0-cp310-cp310-win_amd64.whl (2.6 MB)


In [11]:
import seaborn as sns
import matplotlib.pyplot as plt

# List of columns(continuous variables) for finding skewness
col = ['cbd_dist','min_dist_mrt','floor_area_sqm','lease_remain_years','storey_median','resale_price']

for i in col:
    plt.figure(figsize=(8, 6))  
    sns.boxplot(data=df, x=i)
    plt.title(f'Boxplot of {i}')
    plt.xlabel(i)
    plt.show()

ModuleNotFoundError: No module named 'seaborn'

In [2]:
import pandas as pd 

In [10]:
df = pd.read_csv("J:\Studies\GUVI\projects\capstone\singapore\combined.csv")
df

C:\Users\Joshua Sam\AppData\Local\Temp\ipykernel_9496\2054376662.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("J:\Studies\GUVI\projects\capstone\singapore\combined.csv")


,latitude,longitude,blk_no,road_name,postal_code,address,cbd_dist,min_dist_mrt,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,1.294801,103.854467,1,BEACH ROAD,189673.0,1 BEACH RD,1351.713661,143.728945,2015-01,KALLANG/WHAMPOA,3 ROOM,1,BEACH RD,04 TO 06,74.0,Improved,1979.0,388000.0,63
1,1.294801,103.854467,1,BEACH ROAD,189673.0,1 BEACH RD,1351.713661,143.728945,2016-03,KALLANG/WHAMPOA,3 ROOM,1,BEACH RD,07 TO 09,68.0,Improved,1979.0,350000.0,62
2,1.294801,103.854467,1,BEACH ROAD,189673.0,1 BEACH RD,1351.713661,143.728945,2017-04,KALLANG/WHAMPOA,3 ROOM,1,BEACH RD,07 TO 09,68.0,Improved,1979.0,340000.0,61 years 05 months
3,1.294801,103.854467,1,BEACH ROAD,189673.0,1 BEACH RD,1351.713661,143.728945,2017-05,KALLANG/WHAMPOA,3 ROOM,1,BEACH RD,07 TO 09,68.0,Improved,1979.0,322000.0,61 years 05 months
4,1.294801,103.854467,1,BEACH ROAD,189673.0,1 BEACH RD,1351.713661,143.728945,2017-06,KALLANG/WHAMPOA,3 ROOM,1,BEACH RD,07 TO 09,68.0,Improved,1979.0,370000.0,61 years 03 months
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232290,1.286814,103.828928,9B,BOON TIONG ROAD,163009.0,9B BOON TIONG RD,2525.251132,182.864524,2023-12,BUKIT MERAH,5 ROOM,9B,BOON TIONG RD,19 TO 21,112.0,Improved,2016.0,1400000.0,91 years 02 months
232291,1.286814,103.828928,9B,BOON TIONG ROAD,163009.0,9B BOON TIONG RD,2525.251132,182.864524,2024-03,BUKIT MERAH,4 ROOM,9B,BOON TIONG RD,25 TO 27,92.0,Model A,2016.0,1188000.0,90 years 11 months
232292,1.286814,103.828928,9B,BOON TIONG ROAD,163009.0,9B BOON TIONG RD,2525.251132,182.864524,2024-06,BUKIT MERAH,5 ROOM,9B,BOON TIONG RD,34 TO 36,112.0,Improved,2016.0,1588000.0,90 years 08 months
232293,1.286814,103.828928,9B,BOON TIONG ROAD,163009.0,9B BOON TIONG RD,2525.251132,182.864524,2024-07,BUKIT MERAH,4 ROOM,9B,BOON TIONG RD,10 TO 12,92.0,Model A,2016.0,1100000.0,90 years 07 months


In [4]:
data = pd.read_csv('J:\\Studies\\GUVI\\projects\\capstone\\singapore\\combined.csv', dtype={'column_name': str})


C:\Users\Joshua Sam\AppData\Local\Temp\ipykernel_9496\3659412896.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('J:\\Studies\\GUVI\\projects\\capstone\\singapore\\combined.csv', dtype={'column_name': str})


In [5]:
data = pd.read_csv('J:\\Studies\\GUVI\\projects\\capstone\\singapore\\combined.csv')
print(data.columns)


Index(['latitude', 'longitude', 'blk_no', 'road_name', 'postal_code',
       'address', 'cbd_dist', 'min_dist_mrt', 'month', 'town', 'flat_type',
       'block', 'street_name', 'storey_range', 'floor_area_sqm', 'flat_model',
       'lease_commence_date', 'resale_price', 'remaining_lease'],
      dtype='object')


C:\Users\Joshua Sam\AppData\Local\Temp\ipykernel_9496\955663053.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('J:\\Studies\\GUVI\\projects\\capstone\\singapore\\combined.csv')


In [6]:
data = pd.read_csv('J:\\Studies\\GUVI\\projects\\capstone\\singapore\\combined.csv', low_memory=False)


In [7]:
data = pd.read_csv('J:\\Studies\\GUVI\\projects\\capstone\\singapore\\combined.csv')

# Inspect the problematic column
print(data.iloc[:, 4].unique())  # Replace 4 with the column index/name

# Convert to a consistent type (e.g., numeric, string)
data.iloc[:, 4] = pd.to_numeric(data.iloc[:, 4], errors='coerce')  # Convert to numeric, replace errors with NaN


[189673.0 460001.0 461001.0 ... 312099.0 162009.0 163009.0]


C:\Users\Joshua Sam\AppData\Local\Temp\ipykernel_9496\73156249.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('J:\\Studies\\GUVI\\projects\\capstone\\singapore\\combined.csv')


In [9]:
data = pd.read_csv('J:\\Studies\\GUVI\\projects\\capstone\\singapore\\combined.csv', low_memory=False)
print(data.iloc[:, 4].unique())  # Replace 4 with the column index/name


['189673' '460001' '461001' ... '312099' '162009' '163009']


In [8]:
data = pd.read_csv('J:\\Studies\\GUVI\\projects\\capstone\\singapore\\combined.csv', error_bad_lines=False)


TypeError: read_csv() got an unexpected keyword argument 'error_bad_lines'